In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('datasets/vauxhall.csv')
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Corsa,2018,7885,Manual,9876,Petrol,145,55.4,1.4
1,Corsa,2019,11995,Manual,2500,Petrol,145,54.3,1.4
2,Corsa,2017,9777,Automatic,9625,Petrol,145,47.9,1.4
3,Corsa,2016,8500,Manual,25796,Petrol,30,55.4,1.4
4,Corsa,2019,10000,Manual,3887,Petrol,145,43.5,1.4
...,...,...,...,...,...,...,...,...,...
13627,Tigra,2007,2499,Manual,48640,Petrol,165,46.3,1.4
13628,Vectra,2007,1795,Manual,75000,Petrol,240,39.2,1.8
13629,Vectra,2005,1495,Manual,74000,Petrol,235,38.7,1.8
13630,Vectra,2008,1695,Automatic,131000,Diesel,200,39.8,1.9


In [33]:
df['model'] = df['model'].str.strip()

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13632 entries, 0 to 13631
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         13632 non-null  object 
 1   year          13632 non-null  int64  
 2   price         13632 non-null  int64  
 3   transmission  13632 non-null  object 
 4   mileage       13632 non-null  int64  
 5   fuelType      13632 non-null  object 
 6   tax           13632 non-null  int64  
 7   mpg           13632 non-null  float64
 8   engineSize    13632 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 958.6+ KB


In [35]:
df.isnull().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

## Importing ML Libraries

In [36]:
import numpy as np
import sklearn.metrics as m
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
# cross validation
from sklearn.model_selection import cross_val_score
# grid search
from sklearn.model_selection import GridSearchCV
# outlier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.compose import ColumnTransformer

## Feature Selection

In [37]:
X = df.drop(columns=['price'])
y = df['price']

In [38]:
X

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,Corsa,2018,Manual,9876,Petrol,145,55.4,1.4
1,Corsa,2019,Manual,2500,Petrol,145,54.3,1.4
2,Corsa,2017,Automatic,9625,Petrol,145,47.9,1.4
3,Corsa,2016,Manual,25796,Petrol,30,55.4,1.4
4,Corsa,2019,Manual,3887,Petrol,145,43.5,1.4
...,...,...,...,...,...,...,...,...
13627,Tigra,2007,Manual,48640,Petrol,165,46.3,1.4
13628,Vectra,2007,Manual,75000,Petrol,240,39.2,1.8
13629,Vectra,2005,Manual,74000,Petrol,235,38.7,1.8
13630,Vectra,2008,Automatic,131000,Diesel,200,39.8,1.9


## Column Extraction

In [39]:
bin_cols = ['transmission', 'fuelType']
num_cols = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
cat_cols = ['model']

## Preprocessing Pipeline

In [40]:
# create pipeline
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])
binary_pipeline = Pipeline([
    ('one_hot_encoder', OrdinalEncoder())
])
cat_pipeline = Pipeline([
    ('one_hot_encoder', OneHotEncoder(drop='first'))
])

# create column transformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('binary', binary_pipeline, bin_cols),
    ('cat', cat_pipeline, cat_cols)
])

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('std_scaler',
                                                  StandardScaler())]),
                                 ['year', 'mileage', 'tax', 'mpg',
                                  'engineSize']),
                                ('binary',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OrdinalEncoder())]),
                                 ['transmission', 'fuelType']),
                                ('cat',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OneHotEncoder(drop='first'))]),
                                 ['model'])])

## Outlier Removal
- this help to remove the outliers from the dataset

In [41]:
outlier_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('outlier', LocalOutlierFactor(n_neighbors=20, contamination=0.1))
])

outlier_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('outlier', LocalOutlierFactor(contamination=0.1))])

In [42]:
yhat = outlier_pipeline.fit_predict(X)
X = X[yhat==1].copy()
y = y[yhat==1].copy()

In [43]:
df = pd.concat([X, y], axis=1)

In [44]:
X# cleane data

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,Corsa,2018,Manual,9876,Petrol,145,55.4,1.4
2,Corsa,2017,Automatic,9625,Petrol,145,47.9,1.4
3,Corsa,2016,Manual,25796,Petrol,30,55.4,1.4
4,Corsa,2019,Manual,3887,Petrol,145,43.5,1.4
6,Corsa,2011,Manual,53067,Petrol,145,49.6,1.4
...,...,...,...,...,...,...,...,...
13626,Tigra,2006,Manual,92000,Petrol,265,36.7,1.8
13627,Tigra,2007,Manual,48640,Petrol,165,46.3,1.4
13628,Vectra,2007,Manual,75000,Petrol,240,39.2,1.8
13629,Vectra,2005,Manual,74000,Petrol,235,38.7,1.8


## Model Pipeline

In [45]:
pipeline1 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor())
])
pipeline1

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', DecisionTreeRegressor())])

In [46]:
pipeline2 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])
pipeline2

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', RandomForestRegressor())])

In [47]:
pipeline3 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])
pipeline3

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', LinearRegression())])

In [48]:
pipeline4 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', SVR())
])
pipeline4

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', SVR())])

## Training Model and Evaluation

In [49]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
model1 = pipeline1.fit(train_X, train_y)
yhat = model1.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Decision Tree Regressor")
print(f'MAE: {mae} - tells us how far off our predictions are on average.')
print(f'MSE: {mse} - tells us how far off our predictions are on average squared.')
print(f'RMSE: {rmse} - tells us how far off our predictions are on average squared root.')
print(f'R2: {r2} - tells us how much of the variance in the target variable is explained by the model.')

Decision Tree Regressor
MAE: 879.1568491454061 - tells us how far off our predictions are on average.
MSE: 1654333.503248914 - tells us how far off our predictions are on average squared.
RMSE: 1286.2089656229714 - tells us how far off our predictions are on average squared root.
R2: 0.8645219470327881 - tells us how much of the variance in the target variable is explained by the model.


In [50]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
model2 = pipeline2.fit(train_X, train_y)
yhat = model2.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Random Forest Regressor")
print(f'MAE: {mae} - tells us how far off our predictions are on average.')
print(f'MSE: {mse} - tells us how far off our predictions are on average squared.')
print(f'RMSE: {rmse} - tells us how far off our predictions are on average squared root.')
print(f'R2: {r2} - tells us how much of the variance in the target variable is explained by the model.')

Random Forest Regressor
MAE: 731.1692958734262 - tells us how far off our predictions are on average.
MSE: 1192143.081569789 - tells us how far off our predictions are on average squared.
RMSE: 1091.8530494392498 - tells us how far off our predictions are on average squared root.
R2: 0.9023720288368564 - tells us how much of the variance in the target variable is explained by the model.


In [51]:
model3 = pipeline3.fit(train_X, train_y)
yhat = model3.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Linear Regression")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

Linear Regression
MAE: 1018.306109896984
MSE: 2035676.0499490227
RMSE: 1426.7711974766742
R2: 0.8332927265406516


In [52]:
model4 = pipeline4.fit(train_X, train_y)
yhat = model4.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Support Vector Regression")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

Support Vector Regression
MAE: 2344.4774980729876
MSE: 9747453.477869505
RMSE: 3122.0912026828278
R2: 0.20175344573702647


In [53]:
df['dtree_pred'] = model1.predict(df.drop(columns=['price']))
df['rf_pred'] = model2.predict(df.drop(columns=['price']))
df['lr_pred'] = model3.predict(df.drop(columns=['price']))
df['svr_pred'] = model4.predict(df.drop(columns=['price']))

## Visualization

In [54]:
# histogram
fig = px.histogram(df, x='price', nbins=100, title='Price Distribution')
fig.show()

## Saving the model

In [55]:
import joblib

In [56]:
joblib.dump(model1, 'vauxhall/dtree_model.pkl')
joblib.dump(model2, 'vauxhall/rf_model.pkl')
joblib.dump(model3, 'vauxhall/lr_model.pkl')
joblib.dump(model4, 'vauxhall/svr_model.pkl')

['vauxhall/svr_model.pkl']

In [57]:
X.fuelType.unique()

array(['Petrol', 'Diesel'], dtype=object)

In [58]:
X.model.unique()

array(['Corsa', 'Astra', 'Viva', 'Mokka', 'Mokka X', 'Crossland X',
       'Zafira', 'Meriva', 'Zafira Tourer', 'Adam', 'Grandland X',
       'Antara', 'Insignia', 'GTC', 'Combo Life', 'Vivaro', 'Cascada',
       'Agila', 'Tigra', 'Vectra'], dtype=object)

In [59]:
X.transmission.unique()

array(['Manual', 'Automatic', 'Semi-Auto'], dtype=object)

In [60]:
X

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,Corsa,2018,Manual,9876,Petrol,145,55.4,1.4
2,Corsa,2017,Automatic,9625,Petrol,145,47.9,1.4
3,Corsa,2016,Manual,25796,Petrol,30,55.4,1.4
4,Corsa,2019,Manual,3887,Petrol,145,43.5,1.4
6,Corsa,2011,Manual,53067,Petrol,145,49.6,1.4
...,...,...,...,...,...,...,...,...
13626,Tigra,2006,Manual,92000,Petrol,265,36.7,1.8
13627,Tigra,2007,Manual,48640,Petrol,165,46.3,1.4
13628,Vectra,2007,Manual,75000,Petrol,240,39.2,1.8
13629,Vectra,2005,Manual,74000,Petrol,235,38.7,1.8
